# Analyzing online Blood Bowl matches using Python: API scraping, pandas and plotnine

This blogpost is about **Blood Bowl**, a boardgame I finally started playing last year. The goal of this blog post is to use Python API scraping to fetch online Blood Bowl match outcome data, and to use the `pandas` and `plotnine` Python packages to analyse and visualise the data.

In 2020, a new version of the Blood Bowl board game came out with several changes to the rules, and to the available teams to play with.
Last september, the online Blood Bowl gaming website **FUMBBL** switched to the new ruleset ("BB2020"), with players largely abandoning the previous ruleset (from 2016, "BB2016" hereafter). With a daily game volume of a few hundred matches, I decided it would be interesting to analyse the win rates of the different teams (or "races", given the fantasy world setting), and how these have been impacted by the new ruleset. This is important, because races can get **nerfed** (made weaker) or **buffed** (made stronger) when new versions of the ruleset are released. These changes are common in any gaming community, in an attempt to achieve more balance in the game. We examine whether the current "three tier" system to balance the strength differences between teams is optimal. The results can inform changes that improve balance.


# Python packages used in this blog post

In [ ]:
import random
import time

import requests # API library

import pandas as pd
import numpy as np
import plotnine as p9

#import statsmodels.api as sm # statistics library

# Function for computing confidence intervals
from statsmodels.stats.proportion import proportion_confint   



# Introduction to Blood Bowl

Blood Bowl is a two player game on a board, with playing pieces, like Chess. But instead of two medieval kingdoms fighting, Blood Bowl is about fantasy football, say Tolkien meets rugby. It appealed to me as a teenager (I bought the game in 1994) because it combined the Warhammer playing pieces I liked so much (miniature models "minis" of Orcs, Elves, Dwarves etc) with simple game mechanics, but resulted in complex gameplay. Blood bowl requires a lot of skill to play well, with complex strategic decision making with a lot of uncertainty (heavy dice rolling involved). Blood Bowl is very much alive nowaydays: Over the timespan of a few decennia, an international gaming community has formed around the game, with a players' association, the NAF, with thousands of members, a World championship every two years, and with new editions and models being released on a regular basis. Blood Bowl is not only a game, it is also a sport, [like Chess](https://www.chess.com/article/view/is-chess-a-sport).
And with all sports, statistics is not far away. So lets dive in the world of Blood Bowl stats nerdery.

# Not all teams are created equal: tiers in Blood Bowl

As already mentioned above, Blood Bowl is not balanced with respect to the (30) different team "races" available. For example, it is much harder to win a match playing with a **Vampires** team, compared to playing with an **Orc** team. Not surprisingly, teams that have a higher probability of winning are more popular both in Tabletop tournaments as well as online at FUMBBL. 

According to [this article from the NAF from 2017](https://www.thenaf.net/2017/05/tiers/), already since 2010 efforts were made to balance things out a bit between the different team strengths. For example, the weaker teams get more gold to spend on players, or get more so-called "Star player points" to spend on skilling players up. According to [the NAF](https://www.thenaf.net/tournaments/information/tiers-and-tiering/), traditionally team tiering consists of three groups, with Tier 1 being the strongest teams, and tier 3 the weakest teams. 
 

Below I made a visualization of these three groups, as well as the new BB2020 tiers per the 2020 rulebook. For **Humans** and **Old World Alliance** it is not entirely clear what is the "official tier", as the [current online NAF tournament](https://fumbbl.com/p/group?op=view&group=9298) places them in Tier 2 (same as always), but GW BB2020 places both Humans and Old World Alliance in tier 1, which is kind of weird given that their rules did not change much. So i plotted both.



In [ ]:
race_tiers = pd.read_excel('data/race_tiers_mapping.xlsx',  engine='openpyxl')
race_tiers = race_tiers[ ['race_name', 'bb2020_tier', 'naf_tier']]
race_tiers = race_tiers.dropna()
# format for plotnine
race_tiers_long = pd.melt(race_tiers, id_vars='race_name', value_vars=['bb2020_tier', 'naf_tier'])


In [ ]:
(p9.ggplot(data = race_tiers_long, mapping = p9.aes(x = 'reorder(race_name, value)', y = 'factor(value)', group = 'variable', color = 'variable'))
    + p9.geom_point(size = 5) 
    + p9.coord_flip())
    

Most NAF sanctioned tournaments use some form of tiering, but there exists a lot of variation in how this is implemented.
From the [rules for obtaining NAF sanctioning](https://www.thenaf.net/wp-content/uploads/2020/11/NAF_Tournament_Approval_Document_2021.pdf):

```
Individual rules variations in tournaments are permitted, even encouraged. This is 
in order to give each tournament its individual character.
[...]
Modifications should not radically affect the existing balance between 
races, but incentives may be given to the traditionally less-competitive 
teams, provided this is in moderation. 
```

For example, for the [World Cup in Austria (2019)](http://www.nafworldcup.sbbm-turniere.com/EN/WC4Rules.html), the Tier 2 teams above were further split up, giving four tiers in total. PM example Dutch open 2020

# Analyzing NAF data

Because there is all this variation in tiering, it is difficult to draw conclusions from NAF tournament data.
We can still try though.

Here are the win rates (with a draw counted as half a point) for BB2020 NAF tournaments. These were taken from the [NAF Tableau pages](https://public.tableau.com/app/profile/mike.sann0638.davies/viz/NAFGames_0/Variety) and contain match outcomes of all NAF tournaments since december 2020 using the new ruleset. 
I added uncertainty intervals assuming a simple "coin flip" process with a fixed probability of succes *p*, where i use for *p* the probability of succes that is calculated from the data. This gives us some indication of what variation to expect given the match volumes in the NAF database.

In [ ]:
df_naf = pd.read_csv('data/W_Race_Record_Full_Data_data_bb2020.csv', na_values = '')

df_naf = df_naf.rename(columns={"Race": "race_name", 
                                "Variant": "ruleset_name", 
                                "Win %": "wins"})

# transform wins to 0, 0.5 or 1 numeric
df_naf['wins'] = df_naf['wins'].str.replace('%$', '')
df_naf['wins'] = df_naf['wins'].fillna(0).astype(float).astype(np.int64)
df_naf['wins'] = df_naf['wins']/100

# add tiers
df_naf = pd.merge(df_naf, race_tiers, on='race_name', how='left')

# Tidyverse-like data analysis with Pandas and plotnine

Doing most of my day to day analysis in `R`, i am (by now) used to the tidyverse filosophy of breaking things apart in small steps, with each step on a separate line, and in order of execution. Luckily, the `Pandas` data analysis library allows us to do something similar, here it is called `method chaining`.
The idea is that a Pandas object has methods for all the small operations we want to perform, and the python language that allows chaining these methods together.

Here we demonstrate this by calculating average win percentage by race, and calling this result `perc_win`.


In [ ]:
res = (df_naf
    .groupby(['race_name', 'ruleset_name', 'naf_tier'])
    .agg(        
        perc_win = ('wins', "mean"),
        n_wins = ('wins', "sum"),
        n_games = ('race_name', "count")
    )
    .sort_values( 'n_games', ascending = False)
    .reset_index()) # this adds the group by variables (now index) as a column

res = res.dropna()


res['lower_CI'], res['upper_CI'] =  proportion_confint(
                                      count = round(res['n_wins']).astype(int),
                                      nobs = res['n_games'],
                                      alpha = 0.05
                                  )

(p9.ggplot(data = res, mapping = p9.aes(x = 'reorder(race_name, perc_win)', y = 'perc_win', 
size = 'n_games', group = 'factor(ruleset_name)', color = 'factor(ruleset_name)'))
    + p9.geom_errorbar(p9.aes(ymin = 'lower_CI', ymax = 'upper_CI', color = 'factor(naf_tier)'))
    + p9.geom_point(colour = "black" )
    + p9.scale_size_area() 
    + p9.coord_flip()
    + p9.geom_hline(yintercept = 0.5)
    + p9.ggtitle("raw win rates NAF BB2020 tournaments"))


We can see that, even with most tournaments having some form of tiering, there are still teams that win NAF tournaments more often than others. For example, this quote is from the most recent ["NAF tournament report"](https://www.thenaf.net/rankings/elo-ranking/tableau/the-naf-report-2/):

```
Underworld continued to dominate in October, winning the two largest tournaments and maintaining the greater than 60% win ratio.  This is
generally due to swarming giving them more players on the pitch and the use of Hakflem and Morg'n'Thorg. (N.b. these are special "Star Players" that can be added to teams)
```

This is exactly what we saw above in the NAF data. At the same time, we see that in the NAF tournaments, **Skaven** (A tier 1 team) win equally often as **Ogres**, a tier 3 team. This could very well be because of the tiering systems being used.

However, the best way to learn about the teams relative strength is in the absence of any tiering. Then we can directly interpret the win rates as measuring relative team strength. For this, we turn to online Blood Bowl, and more specific, to **FUMBBL**!

# Blood Bowl online: FUMBBL 

Blood Bowl can also be played online. A paid version called "Blood Bowl 2" with appealing 3D graphics is available on [Steam](https://store.steampowered.com/app/236690/Blood_Bowl_2/). However, a more basic (2D) version is available as [FUMBBL](https://fumbbl.com). It uses a Java client that uploads game results to an online server with an accompanying website that supports the managerial and community aspects of the game (Forming teams, using winnings to buy new players, organizing tournaments, forum discussions etc).
The name **FUMBBL** is likely a wordplay on the combination of fumble (losing the ball in American Football) and BBL which stands for Blood Bowl League.

The **FUMBBL** website (https://fumbbl.com) is one big pile of data. From coach pages, with their teams, to team rosters, with players, and match histories. It's all there.
And the nice thing of **FUMBBL** , for our purpose, is that it has several divisions where all teams start out equal, i.e. there is no tiering system in place.
This allows us to learn what the relative team strengths are purely under the GW BB2020 rules.

To obtain **FUMBBL** data, we need to fetch it match by match, team by team. To do so, the site creator Christer Kaivo-oja, from Sweden, has made an API that allows us to easily fetch data. What follows is a short demonstration how the API works, before we fetch the **FUMBBL** match and team data of the last 12 months.
 

# Behold, the power of Requests / Using Python to fetch FUMBBL data

We use the [Python **Requests** library](https://docs.python-requests.org/en/latest/) to make the API call over HTTPS and obtain the response from the FUMBLL server. The response is in the JSON format, a [light-weight data-interchange format](https://www.json.org/json-en.html) which is both easy to read and write for humans, and easy to parse and generate by computers. So this makes it a natural choice for an API.

Here is an example of what is available at the coach level (in Blood Bowl, people playing the game are called *coaches*, since the playing pieces are already called the *players*). The full documentation of the API can be found at (https://fumbbl.com/apidoc/).


In [ ]:
response = requests.get("https://fumbbl.com/api/coach/teams/gsverhoeven")
# display the complete JSON object {}
response.json()

# Parsing JSON data

Let's have a closer look at the JSON data structure here.
We have a list of key-value pairs. 
Some keys contain simple values, such as `name`, 

In [ ]:
response.json()['name']

but some return as value a new list of key-value pairs, such as `teams`.
Actually this is a "list of "lists of key-value pairs", since we have a separate list for each team.
Even the list of a single team contains new structure, for example under the key `raceLogos`.

In [ ]:
response.json()['teams'][2]['raceLogos']

In [ ]:
response.json()['teams'][2]['name']

# What data do we need? And in what shape?

Now we know how the data comes in, we need to think about which variables we want, and how to structure them.
The most straightforward level to analyze race strength is to look at **match outcomes**.
At its core, the data consists of matches played by teams, commanded by coaches.
Furthermore, we expect race strength to change over time, as new strategies are discovered by the players, or new rules get introduced. So the time dimension is important as well.

So, let's go with a flat data frame with **rows for each match**, and columns for the various variables associated with each match.
These would include:

* Coach ids
* Team races
* Team ids
* Date of the match
* Outcome (Touchdowns of both teams)

With this basic structure, we can add as many match related variables in the future, keeping the basic structure (each row is a match) unchanged.

So lets get the match data!



# Step 1: Getting the match data

So we are mostly intested in the current ruleset, this is `BB2020`. This ruleset became available in **FUMBBL** at september 1st 2021, and two months later, some 5000 games have been played. We also want to compare with the previous ruleset, where we have much more data available. How far do we go back? 
Lets start with matches played during the last year. So starting from september 1st, 2020, up to oktober 1st, 2021. 
This way, we have roughly 12 months of `BB2016` ruleset matches, and one month of `BB2020` matches.

The easiest way to collect match data over a particular period of time is to just loop over `match_id`. The most recent match was 4.334.456, and since rougly 100.000 matches are played each year, we can fiddle about and we find match 4.226.550 played on september 1st, 2020.  So that means we need to collect some 110K matches. 

**VERY IMPORTANT: We do not want to overload the **FUMBBL** server, so we make only three API requests per second. In this way, the server load is hardly affected and it can continue functioning properly for all the Blood Bowl coaches playing their daily games!**

To collect 110K matches, we will need 110000*0.333/3600 = 15 hours.

In [ ]:
# estimated hours fetching data
(4339204-4216257)*0.333/3600

In [ ]:
df_matches = pd.DataFrame(columns=['match_id', 'match_date', 'match_time',  
    'team1_id', 'team1_coach_id', 'team1_roster_id', 'team1_race_name', 'team1_value',
    'team2_id', 'team2_coach_id', 'team2_roster_id', 'team2_race_name', 'team2_value',
    'team1_score', 'team2_score'])

target = 'data/df_matches_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'
print(target)

end_match = 4339204	
begin_match = 4216257 
n_matches = end_match - begin_match
full_run = 0
print(n_matches)

if(full_run):
    for i in range(n_matches):
        api_string = "https://fumbbl.com/api/match/get/" + str(end_match - i)
        # wait 0.5 s on average between each API call
        wait_time = (random.uniform(0.5, 1) + 0.25)/3
        time.sleep(wait_time)
        match = requests.get(api_string)
        match = match.json()
        if match: # fix for matches that do not exist
            match_id = match['id']
            match_date = match['date']
            match_time = match['time']
            team1_id = match['team1']['id']
            team2_id = match['team2']['id']
            team1_score = match['team1']['score']
            team2_score = match['team2']['score']  
            team1_roster_id = match['team1']['roster']['id']
            team2_roster_id = match['team2']['roster']['id']            
            team1_coach_id = match['team1']['coach']['id']
            team2_coach_id = match['team2']['coach']['id']
            team1_race_name = match['team1']['roster']['name'] 
            team2_race_name = match['team2']['roster']['name'] 
            team1_value = match['team1']['teamValue']
            team2_value = match['team2']['teamValue']
            #print(match_id)     
            df_matches.loc[i] = [match_id, match_date, match_time, 
                team1_id, team1_coach_id, team1_roster_id, team1_race_name, team1_value,
                team2_id, team2_coach_id, team2_roster_id, team2_race_name, team2_value,
                team1_score, team2_score]
        else:
            # empty data for this match, create empty row
            match_id = int(end_match - i)
            df_matches.loc[i] = [np.NaN, np.NaN, np.NaN, 
            np.NaN,np.NaN,np.NaN,np.NaN,
            np.NaN,np.NaN,np.NaN,np.NaN,
            np.NaN,np.NaN, np.NaN, np.NaN] # try np.repeat([np.NaN], 13, axis=0) next time
            df_matches.loc[i]['match_id'] = int(match_id)
        if i % 100 == 0: 
            # write tmp data as hdf5 file
            print(i, end='')
            print(".", end='')
            df_matches.to_hdf(target, key='df_matches', mode='w')

    # write data as hdf5 file
    df_matches.to_hdf(target, key='df_matches', mode='w')
else:
    # read from hdf5 file
    #df_matches = pd.read_hdf('data/df_matches_20211102_083311.h5')
    df_matches = pd.read_hdf('data/df_matches_20211106_205843.h5')
#
df_matches.shape

In [ ]:
# convert object dtype columns to proper pandas dtypes datetime and numeric
df_matches['match_date'] = pd.to_datetime(df_matches.match_date) # Datetime object
df_matches['match_id'] = pd.to_numeric(df_matches.match_id) 
df_matches['team1_id'] = pd.to_numeric(df_matches.team1_id) 
df_matches['team1_coach_id'] = pd.to_numeric(df_matches.team1_coach_id) 
df_matches['team1_roster_id'] = pd.to_numeric(df_matches.team1_roster_id) 
df_matches['team2_id'] = pd.to_numeric(df_matches.team2_id) 
df_matches['team2_coach_id'] = pd.to_numeric(df_matches.team2_coach_id) 
df_matches['team2_roster_id'] = pd.to_numeric(df_matches.team2_roster_id) 
df_matches['team1_score'] = pd.to_numeric(df_matches.team1_score) 
df_matches['team2_score'] = pd.to_numeric(df_matches.team2_score) 

# calculate match score difference
df_matches['team1_win'] = np.sign(df_matches['team1_score'] - df_matches['team2_score'])
df_matches['team2_win'] = np.sign(df_matches['team2_score'] - df_matches['team1_score'])

In [ ]:
df_matches.dtypes


In [ ]:
# 123K matches
df_matches

# First analysis: Orcs are most popular!

Which races are the most popular on FUMBLL in the last year?
For this, we need only to count which races were chosen how many times.

However, since each match contains two teams, we need to create a new dataframe `df_races`, double in size, that contains for each row the `match_id` and `team_race` of one of the two teams.

In [ ]:
team1_races = df_matches[['match_id', 'team1_race_name']]
team2_races = df_matches[['match_id', 'team2_race_name']]

# make column names equal
team1_races.columns = team2_races.columns = ['match_id', 'race_name']

# row bind the two dataframes
df_races = pd.concat([team1_races, team2_races])

# aggregate by race_name
res = (df_races
        .groupby(['race_name'])
        .size()
        .reset_index(name='n_games')
)

# select most popular races for filtering
top_races = res.loc[(res.n_games > 1000)]['race_name']



In [ ]:
(p9.ggplot(data = res.loc[res['race_name'].isin(top_races)], mapping = p9.aes(x = 'reorder(race_name, n_games)', y = 'n_games'))
    + p9.geom_point(colour = 'gray') 
    + p9.coord_flip())

So the orcs are most popular! 

# dataprep: df_wins prepare for raw win rates by race

If we want to calculate a win rate for each race, we need to decide what to do with draws.
I've never given the matter much thought, but it turns out other people have!
For example, in football, there is a popular weighting scheme where wins are given 3 points, draws 1 point and losses 0 points.
This is called [three points for a win](https://en.wikipedia.org/wiki/Three_points_for_a_win) . 
In Blood bowl leagues and tournaments, my impression is that this rule is used often as well.
If we want to predict which teams perform best in these settings (have the highest probability of winning a tournament), we need to weigh the match outcomes accordingly.
However, in Blood Bowl, it seems that a 2:1:0 (W / D / L) weighting scheme is most commonly used. 
For example, Mike Davies from the NAF calculates win rate by weighting each win as 1 point, and each draw as 0.5 points (For example, [here](https://public.tableau.com/app/profile/mike.sann0638.davies/viz/NAFGames_0/SuccessBB2020)).
So if we want to compare with others, it makes sense to adapt this scheme as well.
This scheme has the advantage that the weighted average win percentage over all matches is always 50%, creating a nice reference point.


This creates again a dataframe that is double in size, `df_wins` because each match generates two rows.

In [ ]:
# make two copies, one for each team in the match
team1_wins = df_matches[['match_id', 'match_date', 'team1_id', 'team1_race_name', 'team1_value', 'team1_win']].copy()
team2_wins = df_matches[['match_id', 'match_date',  'team2_id', 'team2_race_name', 'team2_value', 'team2_win']].copy()

team1_wins.columns = team2_wins.columns = ['match_id', 'match_date', 'team_id', 'race_name', 'team_value', 'wins']

# combine both dataframes
df_wins = pd.concat([team1_wins, team2_wins])

Now we have a dataset on the match-team level, lets prep the data a bit further.

In [ ]:
df_wins.loc[df_wins['wins'] == 0, 'wins'] = 0.5
df_wins.loc[df_wins['wins'] == -1, 'wins'] = 0

# convert to float
df_wins['wins'] = df_wins['wins'].astype(float)

# convert team value 1100k to 1100 integer and and above / below median (= low / high TV)
df_wins['team_value'] = df_wins['team_value'].str.replace('k$', '')
df_wins['team_value'] = df_wins['team_value'].fillna(0).astype(np.int64)

df_wins['tv_bin'] = pd.cut(df_wins['team_value'], [0, 850, 1150,1450, 1750, float("inf")], labels=['< 850', '1K', '1.3K', '1.6K', '> 1750'])


Lets have a look at our dataset:

In [ ]:
df_wins.query('team_value > 1000 & team_value < 1020')

Great! Almost there. There is still something missing though, we need to know, for all the teams in our matches dataset, in what division or league they are playing, and what version of the rules they use. For these we turn to the API again, to fetch more data.

# Step 2: Fetch data on team division and ruleset

Let grab for all teams in `df_matches` the team **race** and **ruleset**.
There are around 25 "official" races / different teams, such as orcs, elves, humans etc.
Each race has different player types, with different strength, abilities, skills, etc.

A limitation of the API is that it shows the current status of the teams and leagues.  For example, leagues that change their rules, as the NAF who for their latest tournament switched to the new BB2020 ruleset in september 2021, but matches played earlier under this ruleset were played using BB2016 rules. 
So we have to use external information to interpret this data properly.



In [ ]:
# make list of all teams that need to be fetched
team_ids = list(df_wins['team_id'].dropna())

# get unique values by converting to a Python set and back to list
team_ids = list(set(team_ids))

len(team_ids)


So we have to fetch data for 43K different teams. We use the same approach as above, looping over all team_ids and making a separate API call for each team.

**IMPORTANT: here too, we limit ourselves to 3 API calls per second to avoid overloading the FUMBBL server**

In [ ]:
df_teams = pd.DataFrame(columns=['team_id', 'division_id', 'division_name',  'league' ,
    'ruleset', 'roster_id', 'race_name',  'games_played'])

target = 'data/df_teams_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'
print(target)

fullrun = 0

if fullrun:
    print('fetching team data for ', len(team_ids), ' teams')
    for t in range(len(team_ids)):    
        api_string = "https://fumbbl.com/api/team/get/" + str(int(team_ids[t]))
        wait_time = (random.uniform(0.5, 1) + 0.25)/3
        time.sleep(wait_time)
        team = requests.get(api_string)
        team = team.json()
        # grab fields
        team_id = team['id']
        division_id = team['divisionId']
        division_name = team['division']
        ruleset = team['ruleset']
        league = team['league']
        roster_id = team['roster']['id']
        race_name = team['roster']['name']
        games_played = team['record']['games']
        # add to dataframe
        df_teams.loc[t] = [team_id, division_id, division_name, league, ruleset, roster_id, race_name, games_played]
        if t % 100 == 0: 
            # write tmp data as hdf5 file
            print(t, end='')
            print(".", end='')
            df_teams.to_hdf(target, key='df_teams', mode='w')
    
    df_teams.to_hdf(target, key='df_teams', mode='w')
else:
    # read from hdf5 file
    df_teams = pd.read_hdf('data/df_teams_20211030_115137.h5')


df_teams['roster_name'] = df_teams['roster_id'].astype(str) + '_' + df_teams['race_name']

df_teams.shape

In [ ]:
# PM make this an external code list

# Splitting out a few particular leagues
df_teams.loc[(df_teams['league'] == 0) & (df_teams['ruleset'] == 6), 'division_name'] = 'Regular_league'
df_teams.loc[(df_teams['league'] == 9298), 'division_name'] = 'NAF'
df_teams.loc[(df_teams['league'] == 10263), 'division_name'] = 'Secret League'
df_teams.loc[(df_teams['league'] == 10455), 'division_name'] = 'CIBBL'
df_teams.loc[(df_teams['league'] == 14713), 'division_name'] = 'Test Open League BB2020'
df_teams.loc[(df_teams['ruleset'] == 888), 'division_name'] = 'LegaGladio'
df_teams.loc[(df_teams['ruleset'] == 1049), 'division_name'] = 'NAF 7s'

# add column ruleset_name
df_teams['ruleset_name'] = 'bb2016'
df_teams['ruleset_name'] = 'bb2016'
df_teams.loc[df_teams['ruleset'] == 4, 'ruleset_name'] = 'bb2020'
df_teams.loc[df_teams['ruleset'] == 2198, 'ruleset_name'] = 'bb2020'
df_teams.loc[df_teams['ruleset'] == 2198, 'division_name'] = 'SL BB2020'
df_teams.loc[df_teams['division_name'] == 'NAF', 'ruleset_name'] = 'mixed'
df_teams.loc[df_teams['division_name'] == 'LegaGladio', 'ruleset_name'] = 'mixed'

# FUMBBL divisions and rulesets

FUMBBL allows coaches to create their own rulesets to play their own leagues and tournaments with. For example, there is a so-called "Secret League" where coaches can play with "Ninja halflings", of with "Ethereal" spirits etc. 

Since we want the team strength for the official rulesets BB2016 and BB2020, we need to drop the matches that are played under different rules.

Lets have look at the various divisions and leagues, which rulesets are used, and which races are played how often.
We only look at divisions and leagues with a sufficient volume of matches, or otherwise we do not have sufficient statistics for each race.
This gives us 33K teams of 43K, so roughly 75% of all teams created last year.

## Filter on most common rulesets

In [ ]:
(df_teams
    .groupby(['ruleset',  'league', 'division_id', 'division_name', 'ruleset_name'])
    .agg( n_teams = ('ruleset', 'count')
    )
    .sort_values('n_teams', ascending = False)
    .query('n_teams > 150')['n_teams']
    .reset_index()
)

Difference between ruleset 1 and 6: no expensive mistakes and wizards inducements and 5 cards. And few different teams.


* Ruleset 6: League (Also BB2016 with Simyin (monkeys) / Bretonnian / Slann / Khorne)
* Ruleset 2228: NAF-BB2020 (switched during 2021 from BB2016 to BB2020)

PM write post suggesting change to freeze rulesets as soons as matches have been played with them.
And force users to create new rulesets if they want to update / change previous rulesets.



* Ruleset 303: United Open Rules (BB2016, used by the Secret League, it allows a lot of extra teams like Ethereal)
* Ruleset 888: LegaGladio (Italians, likely also switched from BB2016 to BB2020)
* Ruleset 2198: Secret League 2020, SL did it proper and created a new ruleset for BB2020


There are a lot of small leagues being played on FUMBBL. Reasons to play in small local leagues vary.

There are a few larger leagues, that have nation wide scale. 

There are two leagues that are international and have sufficient scale to analyse separately.

This is the NAF tournaments (league 9298), and the Secret League (10263).

## NAF Tournaments: What is with ruleset 2228

Ruleset 2228 appears to be BB2020 + slann + khorne, which the NAF prefers over the GW BB2020 ruleset.
From the roster ids, we can see that only from oktober 2021, the NAF switched to BB2020 teams.

Checked by christer: Does this mean that the API only serves the most recent version of a ruleset, and that rulesets can be changed without requiring a new ruleset id to be created? Correct

They seem to be small scale, but consistent 6 week duration events? lets look up a few matches on FUMBBL.

match 4274272 is within a tournament called SteelBowl 2021 [EU]
The tournament has start /end dates 24 jan 2021. The match was at 29 jan.

The match was a league division game, the team played 6 games in jan/feb/march.
The League is called Online NAF tournaments.
ACtually this is a group 9298 that contains tournaments.
The group uses ruleset 2228.

https://fumbbl.com/p/ruleset?id=2228

Skills are awarded prior to games according to a predetermined list, THAT USES TIERS.


**An important feature of this league is that a tier system has been implemented**
This makes match outcomes in this league less comparable to other divisions on FUMBBL.

BUT: Make sure your custom ruleset is set to Predetermined skill progression.



# So final groupings

* Competitive BB2020

VERSUS

* Ranked BB2016
* Black box BB2016
* standard BB2016 League division (ruleset 6 / league 0)

First we check if we can pool these three groups.




# Dataprep: Merging the match data with the team / ruleset data and team tiers

For each match in the `df_wins` **DataFrame** we can now add the team-level information from `df_teams`.
We also add the team tiers we used for the NAF dataset.

In [ ]:
df_wins = pd.merge(df_wins.drop('race_name', 1), df_teams, on='team_id', how='left')

# 66 empty matches: we drop these
#df_wins.query('match_date != match_date')

df_wins = df_wins.dropna(subset=['match_date'])

# add bb2020 tiers
df_wins = pd.merge(df_wins, race_tiers, on='race_name', how='left')

df_wins['team_id'] = pd.to_numeric(df_wins.team_id) 

# Dataprep: getting the time right

To see time trends, its useful to aggregate the data by week. For this we add a week number for each date, and from this week number, we convert back to a date to get a week_date. This last part is useful for plotting with `plotnine`, as this treats dates in a special way (PM how)
We use the ISO definition of week, this has some unexpected behavior near the beginning / end of each year. 

The data starts in week x of 2020, and stops halfway week 44 in 2021. 


In [ ]:
df_wins['week_number'] = df_wins['match_date'].dt.isocalendar().week


# cannot convert to np int64 becaues of missing values
# cannot convert to 'int64'-dtype NumPy array with missing values. Specify an appropriate 'na_value' for this dtype.
df_wins['week_number'] = df_wins['week_number'].astype('Int64')
#df_wins['week_number'] = df_wins['week_number'].fillna(0).astype(np.int64)

# add year based on match dat
df_wins['year'] = pd.DatetimeIndex(df_wins['match_date']).year

# fix year for  ISO week 53 weeks jan 2021
df_wins.loc[(df_wins['year'] == 2021) & (df_wins['week_number'] == 53), 'year'] = 2020

df_wins['week_year'] = df_wins['year'].astype(str) + '-' + df_wins['week_number'].astype(str)

df_wins['week_date'] = pd.to_datetime(df_wins['week_year'].astype("string") + '-1', format = "%Y-%U-%w")

# manual fix of week date (grrrr)
df_wins.loc[(df_wins['week_date'] ==  '2021-01-04') & (df_wins['week_number'] == 53), 'week_date'] = pd.to_datetime('2020-12-31')


# Analysis: Time series of number of games

Now that we have a date variable for each week, `week_date`, we can use plotnine to plot a nice timer series graph of the total number of games played each week.
The introduction of the new **Competitive** league with BB2020 rules is marked by a vertical red line.

In [ ]:
res = (df_wins
    .loc[(df_wins['week_date'] >= '2020-09-01' ) & (df_wins['week_date'] < '2021-10-25')]
    .groupby(['week_date', 'week_number'])
    .agg(        
        n_games = ('race_name', "count") 
    )

    .reset_index()) # this adds the group by variables (now index) as a column

res['n_games'] = res['n_games']/2 # each games creates two rows in df_wins


In [ ]:
(p9.ggplot(data = res, mapping = p9.aes(x = 'week_date', y = 'n_games', group = '1'))
    + p9.geom_point() 
    + p9.geom_line()
    + p9.expand_limits(y=[0,2000])
    + p9.geom_vline(xintercept = '2021-09-01', color = "red")
    + p9.ggtitle("Total FUMBBL games played in the last year"))



As a check on our work, we compare with the plots that FUMBBL itself provides at https://fumbbl.com/p/stats.

Comparing with the 30-week Statistics plot on the FUMMBBL we can conclude that our dataset for sept 2020/ okt 2021 is complete!
The effect of starting the new BB2020 division is also clearly visible, with the number of games played each week almost doubling.

Not visible on this plot, but clearly visible on the FUMBBL website, is the effect of COVID-19 on online gaming volume.
Starting in march 2020, online gaming activity saw a huge surge. Tabletop leagues switched to online etc. 

What this plot does show is that the COVID-19 effect was declining in 2021, with game volume in july 2021 falling below july 2019 (pre-corona) levels.


# Analysis: Games per week by division

The games on FUMBBL are played within several divisions. Lets do a breakdown by division:





In [ ]:
res = (df_wins
    .loc[(df_wins['week_date'] >= '2020-09-01' ) & (df_wins['week_date'] < '2021-10-25')]
    .groupby(['division_name', 'week_date', 'year'])
    .agg(        
        n_games = ('race_name', "count")
    )
    .reset_index()) # this adds the group by variables (now index) as a column

res['n_games'] = res['n_games']/2


In [ ]:
(p9.ggplot(data = res.query("division_name != 'FFB Test'"), mapping = p9.aes(x = 'week_date', y = 'n_games', 
group = 'factor(division_name)', color = 'factor(division_name)'))
    + p9.geom_point() 
    + p9.geom_line()
    + p9.expand_limits(y=[0,2000])
    + p9.ggtitle("FUMBBL games played in the last year"))


Here it is interesting to compare the *Black box* division with the *League* division.

# Analysis: win rate Blackbox vs Ranked vs League division

Now in the end, we want to compare BB2016 with BB2020. But as we can see above, there are various BB2016 divisions.
Can we just pool them? Or are there differences that are important for our comparison?

Lets first check out the BB2016 divisions.

Does it matter if coaches both need to agree for a match as in the `Ranked` division? 
For the BB2016 rules, we have data for both divisions, one in which coaches can choose their opponent, and one for which they are randomly matched to other teams. 

First, compare Team value distributions between the two divisions, to see what exactly we are comparing.

In [ ]:
bb2016_divisions = ['Ranked', 'Regular_league', 'Blackbox']

(p9.ggplot(data = df_wins[df_wins['division_name'].isin(bb2016_divisions)], mapping = p9.aes(x = 'reorder(division_name, team_value)', y = 'team_value', 
group = 'factor(division_name)', color = 'factor(division_name)'))
    + p9.geom_boxplot()
    #+ p9.scale_size_area() 
    + p9.coord_flip()
    + p9.geom_hline(yintercept = 1100)
    + p9.ggtitle("Team value distributions"))

Clearly, regular league (tournaments) have much lower team value, with a median slightly above 1100, often used for tournaments.
This makes sense, as people tend to play longer with teams outside of leagues / tournaments, which force redrafts and / or are limited in duration.
So, we have to look within TV bins.


In [ ]:
#bb2016_divisions = ['Ranked', 'Regular_league']
bb2016_divisions = ['Ranked', 'Blackbox', 'Regular_league']

tv_bins = ['1K', '1.3K', '1.6K']

res = (df_wins[df_wins['division_name'].isin(bb2016_divisions)]
    .loc[df_wins['tv_bin'].isin(tv_bins)]
    .groupby(['division_name', 'ruleset_name', 'race_name', 'tv_bin'])
    .agg(        
        perc_win = ('wins', "mean"),
        n_wins = ('wins', "sum"),
        n_games = ('race_name', "count")
    )
    .sort_values( 'n_games', ascending = False)
    .reset_index()) # this adds the group by variables (now index) as a column

res = res.dropna()

In [ ]:
res

In [ ]:
import statsmodels.api as sm
# Function for computing confidence intervals
from statsmodels.stats.proportion import proportion_confint   

res['lower_CI'], res['upper_CI'] =  proportion_confint(
                                      count = round(res['n_wins']).astype(int),
                                      nobs = res['n_games'],
                                      alpha = 0.05
                                  )

In [ ]:

(p9.ggplot(data = res.query('n_games > 10 & tv_bin == "1K"'), mapping = p9.aes(x = 'reorder(race_name, perc_win)', y = 'perc_win', 
size = 'n_games', group = 'factor(division_name)', color = 'factor(division_name)'))
    + p9.geom_errorbar(p9.aes(ymin = 'lower_CI', ymax = 'upper_CI'))
    + p9.geom_point(shape = '|', size = 5) # color = 'black', 
    + p9.facet_wrap('tv_bin')
    + p9.scale_size_area() 
    + p9.coord_flip()
    + p9.geom_hline(yintercept = 0.5)
    + p9.ggtitle("raw win rates Ranked vs Regular league"))

In [ ]:
df_wins.query('tv_bin == "1K" & division_name == "Regular_league" & race_name == "Amazon"')

So, if we compare Ranked vs regular league **within Team value bins**, no clear substantial differences appear.
This suggests we can pool both leagues and compare both to the Competitive BB2020 match results.
As long as we use team value bins, like "Taureau Amiral".


*Conclusions for Ranked vs Black box*
For the lower tier teams Ogre, Halfling and Goblin, clear differences can be seen between Ranked and Blackbox. Black box performance is lower.
This suggests that we see the effect of strategically avoiding certain opponents, which is not possible in Blackbox. 

This argument could also explain the lower performance of Amazon at higher TV, which are known to perform better at low TV.
For Chaos Chosen, we see a pattern that could be explained if players in Ranked strategically choose their opponents to skill up the team, which is known to only become competitive at higher team value.

This gives some strength to the argument that the Competitive Division is not a **truly** competitive division, since players can choose their opponents, which is not possible at tournaments, and at random matching environments such as Blackbox was.



# Main analysis: win rate BB2016 vs BB2020

If we want two summary statistics instead of one, we need to use the `agg()` method.
Furthermore, if we want the variable we group over as a regular column in the DataFrame, we need to use `reset_index()`.
Now we have the data ready to make a nice plot.

We are going to do the comparison for the main divisions that use BB2016 and BB2020 rules.
The BB2016 rules are in three divisions: Blackbox, Ranked and regular league.

In [ ]:
res = (df_wins
    .groupby(['division_name', 'ruleset', 'league'])
    .agg(        
        #perc_win = ('wins', "mean"),
        n_games = ('race_name', "count")
    )
    .sort_values( 'n_games', ascending = False)
    .reset_index()) # this adds the group by variables (now index) as a column

res['n_games'] = res['n_games']/2

main_divisions = res.query('n_games > 6000')['division_name']

main_divisions


In [ ]:
res = (df_wins[df_wins['division_name'].isin(main_divisions)]
    #.query('team_value > 800 & team_value < 2500')
    .groupby(['race_name', 'ruleset_name', 'naf_tier', 'tv_bin'])
    .agg(        
        perc_win = ('wins', "mean"),
        n_wins = ('wins', "sum"),
        n_games = ('race_name', "count")
    )
    .query('n_games > 0')
    .reset_index()) # this adds the group by variable (now index) as a column


# Adding credible intervals for the win rates

Comparing the win rates of BB2016 and BB2020 to koadah's dataset on http://fumbbldata.azurewebsites.net/stats.html , we find strong agreement.


In [ ]:
import statsmodels.api as sm
# Function for computing confidence intervals
from statsmodels.stats.proportion import proportion_confint   

res['lower_CI'], res['upper_CI'] =  proportion_confint(
                                      count = round(res['n_wins']).astype(int),
                                      nobs = res['n_games'],
                                      alpha = 0.05
                                  )


In [ ]:

(p9.ggplot(data = res.query('n_games > 10 & tv_bin == "1K" '), mapping = p9.aes(x = 'reorder(race_name, perc_win)', y = 'perc_win', 
size = 'n_games', group = 'factor(ruleset_name)', color = 'factor(ruleset_name)'))
    + p9.geom_errorbar(p9.aes(ymin = 'lower_CI', ymax = 'upper_CI'))
    + p9.geom_point(p9.aes(color = 'factor(naf_tier)') )
    + p9.facet_wrap('tv_bin')
    + p9.scale_size_area() 
    + p9.coord_flip()
    + p9.geom_hline(yintercept = 0.5)
    + p9.ggtitle("raw win rates by ruleset"))

In Tier 1, Amazons and Orcs improved substantially. 
Surprisingly, Underworld denizens are now among the strongest teams around, de

In Tier 3, Halflings and goblins also improved. 

Humans a bit. https://bbtactics.com/bb2020-human-starting-rosters/ Do ppl use halflings with ogres? It appears they do.

Old World Alliance got substantially worse. 
Nurgle slightly worse, 

In [ ]:

(p9.ggplot(data = res.query('n_games > 10 & tv_bin == "1.3K" '), mapping = p9.aes(x = 'reorder(race_name, perc_win)', y = 'perc_win', 
size = 'n_games', group = 'factor(ruleset_name)', color = 'factor(ruleset_name)'))
    + p9.geom_errorbar(p9.aes(ymin = 'lower_CI', ymax = 'upper_CI'))
    + p9.geom_point(p9.aes(color = 'factor(naf_tier)') )
    + p9.facet_wrap('tv_bin')
    + p9.scale_size_area() 
    + p9.coord_flip()
    + p9.geom_hline(yintercept = 0.5)
    + p9.ggtitle("raw win rates by ruleset"))

# Conclusions



Both new teams introduced in BB2020, Imperial nobility and Black orcs, are not the strongest teams around with win rates that are below average.
Slann and daemons of Khorne had no official rules from GW yet, so are not included in the Competitive division that uses BB2020 rules.
Bretonnians have been replaced by Imperial Nobility.

Clearly, the BB2020 results mostly apply to low team value (around 1K +/- 150), as there is too little data on FUMBBL yet at higher team values, because skilling up takes time.
The only clear changes at medium team value (around 1.3K +/-150) are improvements for Goblins and for Underworld Denizens.

Others have come to similar conclusions, for example [this](https://bbtactics.com/tournament-skill-stacking/)  

And check out this ruleset
Project all teams viable
The idea behind this ruleset is to make all teams competitive - while trying to maintain balance.

https://bloodbowl.dk/onewebmedia/All_Teams_Viable_Ruleset.pdf


In [ ]:
df_wins.query('race_name == "Human" & ruleset_name == "bb2020" & wins == 1 & team_value < 1150')['team_id']

# Bonus analysis: is there a learning effect of the BB2020 rules

Answer: no not really. PM do this only for TV 1K bin. 

In [ ]:
# aggregate by year and week number

res = (df_wins.query('division_name == "Competitive"')
    .groupby(['week_number', 'year', 'ruleset', 'race_name'])
    .agg(        
        perc_win = ('wins', "mean"),
        n_wins = ('wins', "sum"),
        n_games = ('race_name', "count")
    )
    .reset_index()) # this adds the group by variables (now index) as a column

res

res['lower_CI'], res['upper_CI'] =  proportion_confint(
                                      count = round(res['n_wins']).astype(int),
                                      nobs = res['n_games'],
                                      alpha = 0.05
                                  )

(p9.ggplot(data = res.query('year > 2020 & week_number > 36 & week_number < 43'), 
    mapping = p9.aes(x = 'week_number', y = 'perc_win', group = 'factor(race_name)', color = 'factor(race_name)'))
    + p9.geom_point(p9.aes(size = 'n_games')) 
    + p9.geom_errorbar(p9.aes(ymin = 'lower_CI', ymax = 'upper_CI'))
    + p9.geom_line()
    + p9.scale_size_area() 
    + p9.facet_wrap('race_name')
    + p9.ggtitle("FUMBBL BB2020 win rates by race and week_nr")
    + p9.theme(figure_size=(16, 8)) 
    + p9.theme(legend_position='none'))